# Targeting with Partial Incentives

Diffusion of influence in social networks is a widely studied phenomenon in contexts that vary from viral marketing to
the broader field of social sciences.

A problem related to this phenomenon, known as the Target Set Selection, is the one to find the minimum set of nodes to
influence in order to cause a cascade in the network, i.e. the minimum number of individuals to influence directly in
order to have the whole network influenced indirectly.

A variant of this problem is the one in which individuals can be given relatively small incentives to make a decision
rather than the full effort needed to actually influence them. This is the Partial Incentives variant and here the goal
is to find incentives assignment that minimizes the total cost of influencing the whole network.

Both of the above problems have already been studied. This project is an attempt to apply and compare two algorithms
known for solving the variant with partial incentives of the problem in a nondeterministic network using the
deferred decision principle, which considers edges of the network to be active or inactive based on some probability.

## Project structure

A few utility modules have been created in [sources](/src) folder to easily interact with the data. Pre-processing
tasks, that consist of binding thresholds to nodes and edges as well as randomly remove some of the latter by coin
tossing, have been modeled in the [preprocessing](/src/preprocessing) module.

The two algorithms used for the test are implemented in the [targeting](/src/targeting) module. The ones picked are
[DiscountFrac](https://arxiv.org/abs/1401.7970) by Demaine et al. and [TPI](https://arxiv.org/abs/1611.08687) by
Cordasco et al.

In [ ]:
import matplotlib
import matplotlib.pyplot as pyplot
import numpy
import snap

from src.preprocessing import data_manager
from src.preprocessing import nodes_manager
from src.preprocessing import edges_manager
from src.preprocessing import edge_pruner

from src.targeting import partial_incentives

%matplotlib inline

## Dataset insights

The dataset chosen for the experiment is [wiki-Vote](/src/data/raw) from the
[SNAP Dataset Collection](https://snap.stanford.edu/data/), a directed graph collected from Wikipedia administrator
elections back in 2008.

The dataset is made of about 7 thousands nodes and more than 100 thousands edges. A more in-depth look can be found
below.

In [ ]:
DATASET = "wiki-Vote"

graph = data_manager.load_graph(f"{DATASET}.txt")

print("Number of nodes: ", graph.GetNodes())
print("Number of edges: ", graph.GetEdges())

print("Maximum degree: ", graph.GetNI(snap.GetMxDegNId(graph)).GetDeg())

print("Diameter (approximate): ", snap.GetBfsFullDiam(graph, 10))
print("Triangles: ", snap.GetTriads(graph))
print("Clustering coefficient: ", snap.GetClustCf(graph))

## Experiments settings

The deferred decision principle is applied by generating 10 different probabilistic graphs for each node threshold-edge
threshold combination.

Node thresholds are set with 3 different approaches: constant, degree-proportional and random. Constants are chosen as
integers between 2 and 10; degree-proportional thresholds are chosen as fractions of the degree of each node, from 0.1
to 0.9 with a 0.1 step; random ones are generated uniformly for each node in the range that goes from 1 to its degree.

Edge thresholds to be used to apply the deferred decision principle are chosen in 2 different ways: randomly and
degree-proportional. Random thresholds are generated uniformly in the range from 0 to 1 for each edge;
degree-proportional ones are applicable only for directed graphs and their value is the inverse of the in-degree of
the node the edge points to.

Overall we have 3 different approaches to set the threshold of the nodes while 2 are the ways to set it on edges. The
total amount of combinations is then 6 while the number of experiments is 19 × 2 × 10 = 380.

Due to long computation times it's better to generate the graphs, bind the thresholds to the nodes, prune some of the
edges by applying the deferred decision principle and store them to be able to load the results whenever they're needed
for analysis.

Also, a maximum budget must be defined as the DiscountFrac algorithm uses it to constraint expenses. Using a very large
value, however, leads the algorithm to influence the whole network using only as much as needed of it.

In [ ]:
# Components to associate filenames to the graph used to generate them
N_MARKER_CONSTANT = "n-constant-"
N_MARKER_PROPORTIONAL = "n-proportional-"
N_MARKER_RANDOM = "n-random-"

E_MARKER_PROPORTIONAL = "e-proportional-"
E_MARKER_RANDOM = "e-random-"

# Number of samples to average results among
SAMPLES = 10

# Node threshold ranges
CONSTANT_THRESHOLD_MIN = 2
CONSTANT_THRESHOLD_MAX = 10

PROPORTIONAL_THRESHOLD_MIN = 0.1
PROPORTIONAL_THRESHOLD_MAX = 0.9
PROPORTIONAL_THRESHOLD_COUNT = 9

# Maximum budget for algorithms that are bounded on the total expense
MAX_BUDGET = 2147483648

## Results

Results are reported for each node-edge threshold combination, so 6 sections follow.

Each section has 2 code cells, one to generate the data and the other to average out and plot the relative results.

Algorithms performances are plotted in graphs with the total cost computed by each on the y-axis while the x-axis is
used for the thresholds. Results of random nodes thresholds however are reported in tables.

### Constant node thresholds with degree-proportional edge thresholds

Here is the case where node thresholds are set as constants while edges activation thresholds are chosen as proportional
to the degree of the node they point to.

In [ ]:
for threshold in range(CONSTANT_THRESHOLD_MIN, CONSTANT_THRESHOLD_MAX + 1):
    for i in range(SAMPLES):
        # Define the file name for the current combination
        name = f"{DATASET} {N_MARKER_CONSTANT}{threshold} {E_MARKER_PROPORTIONAL}{i}"
        
        # Compute nodes and edges thresholds
        node_thresholds = nodes_manager.constant_thresholds(graph, threshold)
        edge_thresholds = edges_manager.degree_proportional_thresholds(graph)
        
        # Prune edges
        probabilistic = edge_pruner.random_prune(graph, edge_thresholds, i)
        
        # Dump the graph generated and its thresholds
        data_manager.store_graph(probabilistic, f"{name}.graph")
        data_manager.store_node_thresholds(node_thresholds, f"{name}.node.thresholds")
        
        # Compute incentive assignment for each node
        tpi_incentives = partial_incentives.tpi(probabilistic, node_thresholds)
        data_manager.store_incentives(tpi_incentives, f"{name}.tpi.incentives")
        
        discount_frac_incentives = partial_incentives.discount_frac(probabilistic, node_thresholds, MAX_BUDGET)
        data_manager.store_incentives(discount_frac_incentives, f"{name}.discount.incentives")

Once the files are saved it's possible to see the results by plotting the average cost obtained for the 10 different
ways the edges of the original graph have been pruned when applying the deferred decision principle.

In [ ]:
thresholds = range(CONSTANT_THRESHOLD_MIN, CONSTANT_THRESHOLD_MAX + 1)
tpi = numpy.zeros(CONSTANT_THRESHOLD_MAX - CONSTANT_THRESHOLD_MIN + 1)
discount = numpy.zeros(CONSTANT_THRESHOLD_MAX - CONSTANT_THRESHOLD_MIN + 1)

for threshold in range(CONSTANT_THRESHOLD_MIN, CONSTANT_THRESHOLD_MAX + 1):
    # Track the sum for each threshold
    tpi_total = 0
    discount_total = 0
    
    for i in range(SAMPLES):
        # Define the file name for the current combination
        name = f"{DATASET} {N_MARKER_CONSTANT}{threshold} {E_MARKER_PROPORTIONAL}{i}"
        
        # Load incentives
        tpi_incentives = data_manager.load_incentives(f"{name}.tpi.incentives")
        discount_incentives = data_manager.load_incentives(f"{name}.discount.incentives")
        
        # Compute the total for each algorithm
        tpi_total += sum(tpi_incentives.values())
        discount_total += sum(discount_incentives.values())
    
    # Compute the average
    tpi[threshold - CONSTANT_THRESHOLD_MIN] = tpi_total / SAMPLES
    discount[threshold - CONSTANT_THRESHOLD_MIN] = discount_total / SAMPLES

# Plot the results
figure, axes = pyplot.subplots()

figure.suptitle("Constant node thresholds with degree-proportional edge thresholds")
axes.set_xlabel("Threshold")
axes.set_ylabel("Cost")

axes.plot(thresholds, tpi, "r", label="TPI")
axes.plot(thresholds, discount, "b", label="DiscountFrac")

pyplot.show()

### Constant node thresholds with random edge thresholds

### Degree-Proportional node thresholds with degree-proportional edge thresholds

### Degree-Proportional node thresholds with random edge thresholds

### Random node thresholds with degree-proportional edge thresholds

### Random node thresholds with random edge thresholds